In [2]:
#Imports
import os
import sys
import glob
import torch
import torchvision

import numpy    as np
import datetime as dt
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot   as plt

from PIL               import Image
from torch.utils.data  import Dataset
from torch.autograd    import Variable
from torch.optim       import lr_scheduler

from torch.utils.data  import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision       import transforms, datasets, models
from os                import listdir, makedirs, getcwd, remove
from os.path           import isfile, join, abspath, exists, isdir, expanduser


%matplotlib inline

****Reading the Dataset****

In [3]:
data_path = "../input/ammi-2021-convnets/"
train_path = join(data_path, "train/train")
test_path = join(data_path,"test/test")
extraimage_path = join(data_path, "extraimages/extraimages")

****Preprocessing****

In [19]:
# Transformations for both the training and testing data
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
normalize = transforms.Normalize(mean=mean, std=std)



train_transforms = transforms.Compose([
                                       transforms.RandomResizedCrop(500),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.ToTensor(),
                                      normalize])

test_transforms = transforms.Compose([ 
                                      transforms.Resize((500, 500)),
                                       transforms.ToTensor(),
                                       normalize])



In [20]:
class CassavaDataset(Dataset):
    def __init__(self, path, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
        self.file_list = files
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
            
        return im.view(3, 500, 500), classCategory, fileName

In [21]:
train_data = CassavaDataset(train_path, transform=train_transforms)
test_data = CassavaDataset(test_path, transform=test_transforms)

****Split train data to train and valid****

In [22]:
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

****Data loader****

In [23]:
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


train_loader = torch.utils.data.DataLoader(train_data, batch_size=16,
                                             sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=16,
                                             sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=16)

****Device configuration****

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

****Model****

In [10]:
# model = torchvision.models.resnet18(pretrained=True)
# model = torchvision.models.resnet50(pretrained=True)
# model = torchvision.models.resnet101(pretrained=True)

model = torchvision.models.resnext50_32x4d(pretrained=True)
#model = torchvision.models.resnext101_32x8d(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

In [11]:
# for param in model.parameters():
            
#     param.requires_grad = False

In [12]:
# change the Dimensions of the last layer 
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)

****Train_Test Function****

In [13]:
def train_test(model, criterion, train_loader,val_loader, optimizer, num_epochs, best_acc= 0):
    """Simple training loop for a PyTorch model.""" 
    
    
    # Move model to the device (CPU or GPU).
    model.to(device)
    
    best_accur = best_acc
    best_acc_epoch = 0
    # Exponential moving average of the loss.
    ema_loss = None

    print('----- Training Loop -----')
    # Loop over epochs.
    for epoch in range(num_epochs):
        
        # Make sure model is in training mode.
        model.train()
        
        correct_train = 0
        # Loop over data.
        for batch_idx, (features, target,_) in enumerate(train_loader):
            
            # Forward pass.
            output = model(features.to(device))
            loss = criterion(output.to(device), target.to(device))

              # Backward pass.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
          # NOTE: It is important to call .item() on the loss before summing.
            if ema_loss is None:
                ema_loss = loss.item()
            else:
                ema_loss += (loss.item() - ema_loss) * 0.01 

            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)

            # Count number of correct predictions.
            correct_train += pred.cpu().eq(target.view_as(pred)).sum().item()

        # Print out progress the end of epoch.
        
        print('Epoch: {} \tLoss: {:.6f}'.format(epoch, ema_loss),)
        

        """Measures the accuracy of a model on a validation."""
    
        # Make sure the model is in evaluation mode.
        model.eval()
    
        correct_test = 0
        
        #print('----- Model Evaluation -----')
    
        # We do not need to maintain intermediate activations while testing.
        
        with torch.no_grad():

          # Loop over test data.
          for features, target,_ in val_loader:

                # Forward pass.
                output = model(features.to(device))

                # Get the label corresponding to the highest predicted probability.
                pred = output.argmax(dim=1, keepdim=True)

                # Count number of correct predictions.
                correct_test += pred.cpu().eq(target.view_as(pred)).sum().item()
      
        # Print test accuracy.
        percent_train = 100. * correct_train / (len(train_loader.sampler) )
        print(f'Train accuracy: {correct_train} / {len(train_loader.sampler) } ({percent_train:.0f}%)')
        
        percent_test = 100. * correct_test / (len(val_loader.sampler) )
        print(f'Test accuracy: {correct_test} / {len(val_loader.sampler) } ({percent_test:.0f}%)')
        
        print("")
        if(percent_test > best_accur):
            
            torch.save(model, 'model.ckpt')
            
            best_accur = percent_test
            best_acc_epoch = epoch
        scheduler.step()
            
    return best_accur, best_acc_epoch


In [14]:
# def train(model, criterion, data_loader, optimizer, num_epochs):
#     """Simple training loop for a PyTorch model.""" 
    
#     # Make sure model is in training mode.
#     model.train()
    
#     # Move model to the device (CPU or GPU).
#     model.to(device)
    
#     # Exponential moving average of the loss.
#     ema_loss = None

#     print('----- Training Loop -----')
#     # Loop over epochs.
#     for epoch in range(num_epochs):
        
#       # Loop over data.
#       for batch_idx, (features, target,_) in enumerate(data_loader):
            
#           # Forward pass.
#         output = model(features.to(device))
#         loss = criterion(output.to(device), target.to(device))

#           # Backward pass.
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#       # NOTE: It is important to call .item() on the loss before summing.
#         if ema_loss is None:
#             ema_loss = loss.item()
#         else:
#             ema_loss += (loss.item() - ema_loss) * 0.01 

#       # Print out progress the end of epoch.
#       print('Epoch: {} \tLoss: {:.6f}'.format(epoch, ema_loss),)
  

In [14]:
# def test(model, data_loader):
#     """Measures the accuracy of a model on a data set.""" 
#     # Make sure the model is in evaluation mode.
#     model.eval()
#     correct = 0
#     print('----- Model Evaluation -----')
#     # We do not need to maintain intermediate activations while testing.
#     with torch.no_grad():
        
#         # Loop over test data.
#         for features, target,_ in data_loader:
          
#             # Forward pass.
#             output = model(features.to(device))
            
#             # Get the label corresponding to the highest predicted probability.
#             pred = output.argmax(dim=1, keepdim=True)
            
#             # Count number of correct predictions.
#             correct += pred.cpu().eq(target.view_as(pred)).sum().item()

#     # Print test accuracy.
#     percent = 100. * correct / (len(data_loader.sampler))
#     print(f'Test accuracy: {correct} / {len(data_loader.sampler)} ({percent:.0f}%)')
#     torch.save(model.state_dict(), 'model.ckpt')
#     return percent

****Loss function and Optimizer****

In [15]:
criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 7)


****Training****

In [16]:
epochs = 10
train_test(model, criterion, train_loader,valid_loader, optimizer, num_epochs = epochs, best_acc= 0)

----- Training Loop -----
Epoch: 0 	Loss: 0.627644
Train accuracy: 3539 / 4525 (78%)
Test accuracy: 949 / 1131 (84%)

Epoch: 1 	Loss: 0.485048
Train accuracy: 3780 / 4525 (84%)
Test accuracy: 1008 / 1131 (89%)

Epoch: 2 	Loss: 0.398012
Train accuracy: 3964 / 4525 (88%)
Test accuracy: 1001 / 1131 (89%)

Epoch: 3 	Loss: 0.362393
Train accuracy: 3996 / 4525 (88%)
Test accuracy: 995 / 1131 (88%)

Epoch: 4 	Loss: 0.313265
Train accuracy: 4067 / 4525 (90%)
Test accuracy: 1017 / 1131 (90%)

Epoch: 5 	Loss: 0.253910
Train accuracy: 4138 / 4525 (91%)
Test accuracy: 1017 / 1131 (90%)

Epoch: 6 	Loss: 0.228346
Train accuracy: 4188 / 4525 (93%)
Test accuracy: 1026 / 1131 (91%)

Epoch: 7 	Loss: 0.208281
Train accuracy: 4199 / 4525 (93%)
Test accuracy: 1023 / 1131 (90%)

Epoch: 8 	Loss: 0.215263
Train accuracy: 4205 / 4525 (93%)
Test accuracy: 1027 / 1131 (91%)

Epoch: 9 	Loss: 0.234493
Train accuracy: 4188 / 4525 (93%)
Test accuracy: 1022 / 1131 (90%)



(90.80459770114942, 8)

****Submission****

In [26]:
model = torch.load("./model.ckpt")

In [27]:
# Make submission here
model.eval()
model.to(device)
result = []
file_names = []
with torch.no_grad():
    for batch,_ ,names in test_loader:
        output = model(batch.cuda()) 
        result.append(output.argmax(dim=1, keepdim=True))
        file_names.append(names)


import pandas as pd
sub = pd.read_csv("../input/ammi-2021-convnets/sample_submission_file.csv")
print(sub.head())
r = []
f = []
for i, res in enumerate(result):
    for j, res_1 in enumerate(res):
        r.append(train_data.classes[res_1])
        f.append(file_names[i][j].split('/')[-1])

sub['Category'] = r
sub['Id'] = f


  Category              Id
0     cbsd  test-img-0.jpg
1      cmd  test-img-1.jpg
2      cbb  test-img-2.jpg
3      cmd  test-img-3.jpg
4     cbsd  test-img-4.jpg


****Save the submission file****

In [29]:
sub.to_csv("sub.csv", index=False)